In [2]:
from convokit import Corpus, download
# https://convokit.cornell.edu/documentation/stack_politeness.html for mor details
corpus_wiki = Corpus(filename=download("wikipedia-politeness-corpus"))

Dataset already exists at /Users/yuka/.convokit/saved-corpora/wikipedia-politeness-corpus


In [3]:
corpus_stack = Corpus(filename=download("stack-exchange-politeness-corpus"))

Dataset already exists at /Users/yuka/.convokit/saved-corpora/stack-exchange-politeness-corpus


In [4]:
for utt in corpus_stack.iter_utterances():
    print(utt.meta['Normalized Score'])
    print(utt.meta["Annotations"])
    break

0.21732585796906329
{'A33SMNMTMIOJ6T': np.int64(12), 'A2OXXHGAM7B0Y': np.int64(16), 'A28TXBSZPWMEU9': np.int64(15), 'A3EJ5TT2ZGBIDA': np.int64(16), 'A3OY0OL2M0HUTT': np.int64(13)}


In [12]:
print(len(list(corpus_wiki.iter_utterances())))

4353


In [7]:

import json
from google import genai
from google.genai import types
import os
import pandas as pd
import random
from dotenv import load_dotenv
load_dotenv()
client = genai.Client(
    api_key=os.getenv("google_api_key"),
)
sentence = ""
responses = []
for index,utt in enumerate(corpus_wiki.iter_utterances()):
    sentence += f"{index+1}. " + utt.text
    sentence += "\n"
    if index%30==29:
        response = client.models.generate_content(
            model="gemini-3-flash-preview",
            contents="For each of these sentences, output a list of True/False. True, if the sentence contains a modifier, and false if it does not. Output your answer in the form of a list (e.g. [True, False ... ]). Output nothing else:\n"+sentence
        )     
        responses.append(response.text)
        print(index)
        sentence = ""
response = client.models.generate_content(
            model="gemini-3-flash-preview",
            contents="For each of these sentences, output a list of True/False values corresponding to whether it contains a modifier in the form of a list (e.g. [True, False ... ]):\n"+sentence
        )     
sentence = ""
responses.append(response.text)


In [201]:
requests = []
with open("label-modifier-stack.jsonl", "w") as f:
    requests = []
    sentence = ""
    for index,utt in enumerate(corpus_stack.iter_utterances()):
        sentence += f"{index%30+1}. " + utt.text
        sentence += "\n"
        if index%30==29:
            prompt = f"""For each of these sentences, output whether the sentence contains a downtoner. Output True, if the sentence contains a downtoner, and false if it does not. If true, also output the modifier i the sentnece. Output your answer in the format of a json file, for example,\n{{"sentence_1": (False, "NA"), "sentence_2":(True, "a bit"), "sentence_3":(True,"really").....}}
Here are the sentences:\n\n{sentence}\n\nMake sure the second argument (e.g. "only" or "NA") is in double quotes"""  
            requests.append({"key": f"request-{len(requests) + 1}-1", "request": {"contents": [{"parts": [{"text": prompt}]}]}})
            requests.append({"key": f"request-{len(requests) + 1}-2", "request": {"contents": [{"parts": [{"text": prompt}]}]}})
            requests.append({"key": f"request-{len(requests) + 1}-3", "request": {"contents": [{"parts": [{"text": prompt}]}]}})
            sentence = ""
            print(prompt)
    for req in requests:
        f.write(json.dumps(req) + "\n")
uploaded_file = client.files.upload(
    file='label-modifier-stack.jsonl',
    config=types.UploadFileConfig(display_name='label-modifier-stack', mime_type='jsonl')
)
input_file = uploaded_file.name
file_batch_job = client.batches.create(
    model="gemini-3-flash-preview",
    src=input_file,
    config={
        'display_name': "label-modifier-stack",
    },
)

print(f"Created batch job: {file_batch_job.name}")

For each of these sentences, output whether the sentence contains a downtoner. Output True, if the sentence contains a downtoner, and false if it does not. If true, also output the modifier i the sentnece. Output your answer in the format of a json file, for example,
{"sentence_1": (False, "NA"), "sentence_2":(True, "a bit"), "sentence_3":(True,"really").....}
Here are the sentences:

1. Can you explain more in detail, what should I sddelat? Allocate a job? How?
2. Will expressions always be unambiguously parenthesized like that, so you don't have to deal with precedence? Or is `X OR Y AND Z` allowed?
3. how are you resolving function pointers? I am also writing similar kind of program but still wondering how i will resolve function pointers?
4. What is the definition of `buffer`? Is it a local variable?
5. Is `A` a global variable?  What is x?
6. This is a very confusing question!  How are you compiling your C programs?
7. Why not using `isnan()` from math.h? Any specially reason?
8. 

In [62]:
batch_job = client.batches.get(name='batches/2428hko4sbuw71uk8xdvn0brcavc5ytt5dwe')
import ast
import re
import pandas as pd

modifier_list = []
result_file_name = batch_job.dest.file_name
file_content = client.files.download(file=result_file_name).decode('utf-8')

# Dictionary to track sentences we've already added (to avoid duplicates)
seen_sentences = {}

for line_num, line in enumerate(file_content.split('\n'), 1):
    line = line.strip()
    if not line:
        continue
    try:
        data = json.loads(line)
        response = data['response']['candidates'][0]['content']['parts'][0]['text']
        request_key = data['key']
        response = response.strip()
        response = response.replace("(","[").replace(")","]").replace("F","f").replace("T","t").replace(",NA", ",\"NA\"").replace(" NA", "\"NA\"")
        start = response.find('{')
        end = response.rfind('}') + 1
        response = json.loads(response[start:end])
        
        # Process each sentence in the response
        for sentence_key in response.keys():
            is_modifier = response[sentence_key][0]
            modifier_word = response[sentence_key][1]
            
            # Convert string booleans to actual booleans
            if is_modifier in ["true", 'true', True]:
                is_modifier = True
            else:
                is_modifier = False
            
            # Create a unique key for this sentence (request_key + sentence_key)
            unique_key = f"{request_key}_{sentence_key}"
            
            # Only add if we haven't seen this exact sentence before
            if unique_key not in seen_sentences:
                modifier_list.append([is_modifier, modifier_word, request_key[:-2]])
                seen_sentences[unique_key] = True
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON on line {line_num}: {e}")
        continue

df = pd.DataFrame(modifier_list, columns=['is_modifier', 'modifier_word', 'key'])

Error decoding JSON on line 45: Expecting ',' delimiter: line 3 column 26 (char 63)
Error decoding JSON on line 60: Expecting ',' delimiter: line 3 column 26 (char 72)
Error decoding JSON on line 232: Expecting ',' delimiter: line 2 column 26 (char 27)
Error decoding JSON on line 291: Expecting ',' delimiter: line 2 column 26 (char 27)
Error decoding JSON on line 353: Expecting ',' delimiter: line 2 column 26 (char 27)
Error decoding JSON on line 355: Expecting ',' delimiter: line 2 column 26 (char 27)
Error decoding JSON on line 418: Expecting ',' delimiter: line 2 column 26 (char 27)


In [65]:

scenario_list = []

with open("convoKit/label-modifier.jsonl", "r") as f:
    for line_num,line in enumerate(f):
        data =json.loads(line)
        scenario_list.append([data['request']['contents'][0]['parts'][0]['text'],data['key'][:-2]])
scenario_df = pd.DataFrame(scenario_list, columns=['scenario','key'])
df = df.merge(scenario_df, on='key', how='left')

In [66]:
df

,is_modifier,modifier_word,key,scenario
0,False,NA,request-101,"For each of these sentences, output whether th..."
1,True,to say the least,request-101,"For each of these sentences, output whether th..."
2,False,NA,request-101,"For each of these sentences, output whether th..."
3,False,NA,request-101,"For each of these sentences, output whether th..."
4,False,NA,request-101,"For each of these sentences, output whether th..."
...,...,...,...,...
12834,False,NA,request-300,"For each of these sentences, output whether th..."
12835,False,NA,request-300,"For each of these sentences, output whether th..."
12836,False,NA,request-300,"For each of these sentences, output whether th..."
12837,False,NA,request-300,"For each of these sentences, output whether th..."


In [67]:
def extract_sentence_by_newline(scenario_text, row_index):
    """Extract sentence based on row index using newline splits"""
    # Find where sentences start
    sentences_start = scenario_text.find('1. ')
    if sentences_start == -1:
        return ''
    
    # Get just the sentences part
    sentences_section = scenario_text[sentences_start:]
    
    # Split by newlines
    lines = sentences_section.split('\n')
    
    # Filter out empty lines
    lines = [line for line in lines if line.strip()]
    
    # Get the line at row_index
    if row_index < len(lines):
        # Remove the number prefix (e.g., "12. ")
        line = lines[row_index]
        # Remove leading number and dot
        cleaned = re.sub(r'^\d+\.\s+', '', line)
        return cleaned
    
    return ''

# Apply to DataFrame
df['sentence_text'] = df.apply(
    lambda row: extract_sentence_by_newline(row['scenario'], row.name%30),
    axis=1
)


In [70]:
# list unique sentences with newline
df[df.modifier_word == "a bit"].sentence_text.unique().tolist() # list unique sentences with newline as a list

['Hey, you seem to know a bit about coding, so I figured you might have an answer here. If I want use code to grab the target of a page known to be a redirect, how would I do that?',
 'Probably I would have to do quite a bit of alteration in wording. Is that O.K?',
 "Looks like you've been a bit inactive lately, but on the off-chance you check in soon, take a look at <url> and <url>.  You're probably the most familiar with CoM's style and brand of hostility, do you think Freakshownerd here is his sock?",
 "Hey Dan, I see you around but we haven't talked in a bit. What are you up to these days?",
 'Converting <url> to a redirect without preserving either the information or sources, & without discussion was ill-advised. A bit more care in future?',
 "In fact, what I'd like to propose here is that we take a bit more control, establish some stricter edit warring restrictions, unprotect the page, slap up an edit notice, and enforce them with blocks so that people can get back to writing the

In [45]:

df[df.is_modifier == True].to_csv("wiki_filtered_polite.csv")
df = df[df.is_modifier == True]
df.sentence_text[df.modifier_word=="a bit"]

2726      Hey, you seem to know a bit about coding, so I...
2756      Hey, you seem to know a bit about coding, so I...
2786      Hey, you seem to know a bit about coding, so I...
2816      Hey, you seem to know a bit about coding, so I...
2846      Hey, you seem to know a bit about coding, so I...
                                ...                        
125822    I am reluctant to mention this, but having you...
125852    I am reluctant to mention this, but having you...
125882    I am reluctant to mention this, but having you...
125912    I am reluctant to mention this, but having you...
125942    I am reluctant to mention this, but having you...
Name: sentence_text, Length: 2773, dtype: object

In [36]:
with pd.option_context('display.max_rows', None, 
                       'display.max_colwidth', None):
    print(df.sentence_text[df.modifier_word=="a bit"].to_string(index=False))

                                                                                                                                                                                                                                                                                           Hey, you seem to know a bit about coding, so I figured you might have an answer here. If I want use code to grab the target of a page known to be a redirect, how would I do that?
                                                                                                                                                                                                                                                                                           Hey, you seem to know a bit about coding, so I figured you might have an answer here. If I want use code to grab the target of a page known to be a redirect, how would I do that?
                                                                            